In [ ]:
import sys
import os
import torch
from torch import nn
sys.path.append(os.path.abspath('../python'))

cuda = torch.cuda.is_available()
dtype = torch.float
from PIL import Image
from imageProcess import toTorch, readFile, toOutput
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
show = lambda im: Image.fromarray(toOutput(8)(im).transpose(1, 2, 0))
readFile = readFile()

In [ ]:
from moire_obj import Net
m = Net()

In [ ]:
import re
from functools import reduce
getRoot = lambda w, r: tuple(filter(lambda s: s.startswith(r), w.keys()))
replaces = lambda w, r, p, s: list((t, t.replace(p, s)) for t in getRoot(w, r + p))
inserts = lambda w, r, p, s: list((t, t.replace(r + p, r + s + p)) for t in getRoot(w, r + p))
getMatch = lambda o: o.group(1) if len(o.groups()) else o.group(0)
def find(names, r):
    r = re.compile(r)
    return set(map(getMatch, filter(None, map(r.match, names))))
findRs = lambda w, rs: reduce(set.union, (find(tuple(w.keys()), r) for r in rs))
def changeName(w, old, new):
    if new:
        w[new] = w[old]
    del w[old]
def changeNames(w, names):
    for old, new in names:
        changeName(w, old, new)
pf = lambda f, w, r: lambda p, s: [] if p is None else f(w, r, p, s)
getNames = lambda w: lambda rst: reduce(lambda a, r: a + pf(replaces, w, r)(rst[1], rst[2]) + pf(inserts, w, r)(rst[3], rst[4]), findRs(w, rst[0]), [])
cc = lambda w: lambda rsts: tuple(changeNames(w, getNames(w)(rst)) for rst in rsts)
removeRoot = lambda w, r: tuple(changeName(w, old, None) for old in getRoot(w, r))

In [ ]:
modelPath = '../model/demoire/moire_obj.pth'
weights = torch.load(modelPath, map_location='cpu')

getW = getNames(weights)
ccW = cc(weights)

In [ ]:
#modelPath = 'moire_t.pth'
torch.save(weights, modelPath, pickle_protocol=4)

In [ ]:
weights = torch.load('moire_t.pth', map_location='cpu')

In [ ]:
list(weights.keys())

In [ ]:
rsts = [
    ((r'(to_clean1\.)se\.',), None, None, 'se.', 'residual.0.'),
    ((r'(to_clean1\.)gff\.',), None, None, 'gff.', 'residual.0.'),
    ((r'(to_clean1\.)relu\.',), None, None, 'relu.', 'residual.0.'),
    ((r'branch.\.cats\.',), 'relu1.', 'ms.0.0.', None, None),
    ((r'branch.\.convt_F(18|19|20)\.',), 'relu2.', 'ms.1.0.', None, None),
    ((r'branch.\.convt_F(18|19|20)\.',), 'relu3.', 'ms.2.0.', None, None),
    ((r'branch.\.convt_F(18|19|20)\.',), 'conv1.', 'ms.0.1.', None, None),
    ((r'branch.\.convt_F(18|19|20)\.',), 'conv2.', 'ms.1.1.', None, None),
    ((r'branch.\.convt_F(18|19|20)\.',), 'conv3.', 'ms.2.1.', None, None),
    ((r'down2_.\.block\.', r'branch.\.convt_F..\.'), 'ca1.', '0.ca.', None, None),
    ((r'down2_.\.block\.', r'branch.\.convt_F..\.'), 'ca2.', '1.ca.', None, None),
    ((r'down2_.\.block\.', r'branch.\.convt_F..\.'), 'relu1.', '0.relu.', None, None),
    ((r'down2_.\.block\.', r'branch.\.convt_F..\.'), 'relu2.', '1.relu.', None, None),
    ((r'down2_.\.block\.', r'branch.\.convt_F..\.'), 'conv1.', '0.conv1.', None, None),
    ((r'down2_.\.block\.', r'branch.\.convt_F..\.'), 'conv2.', '0.conv2.', None, None),
    ((r'down2_.\.block\.', r'branch.\.convt_F..\.'), 'conv3.', '1.conv1.', None, None),
    ((r'down2_.\.block\.', r'branch.\.convt_F..\.'), 'conv4.', '1.conv2.', None, None),
    ((r'down2_.\.block\.[01]\.', r'branch.\.convt_F..\.[01]\.'), None, None, '', '0.'),
    ((r'branch.\.cats\.',), None, None, 'conv_du.', '0.'),
    ((r'branch.\.cats\.',), 'trans.', '1.', None, None),
    ((r'(branch.\.)cats\.',), 'cats.', 'deepF.0.', None, None),
    ((r'(branch.\.)convt_F01\.',), 'convt_F01.', 'shallowF.0.', None, None),
    ((r'(branch.\.)convt_F02\.',), 'convt_F02.', 'shallowF.1.', None, None),
    ((r'(branch.\.)convt_F03\.',), 'convt_F03.', 'shallowF.2.', None, None),
    ((r'(branch.\.)convt_F04\.',), 'convt_F04.', 'shallowF.3.', None, None),
    ((r'(branch.\.)convt_F05\.',), 'convt_F05.', 'shallowF.4.', None, None),
    ((r'(branch1\.)convt_F11\.',), 'convt_F11.', 'deepF.1.', None, None),
    ((r'(branch1\.)convt_F12\.',), 'convt_F12.', 'deepF.2.', None, None),
    ((r'(branch1\.)convt_F13\.',), 'convt_F13.', 'deepF.3.', None, None),
    ((r'(branch1\.)convt_F14\.',), 'convt_F14.', 'deepF.4.', None, None),
    ((r'(branch1\.)convt_F15\.',), 'convt_F15.', 'deepF.5.', None, None),
    ((r'(branch1\.)convt_F16\.',), 'convt_F16.', 'deepF.6.', None, None),
    ((r'(branch1\.)convt_F17\.',), 'convt_F17.', 'deepF.7.', None, None),
    ((r'(branch1\.)convt_F18\.',), 'convt_F18.', 'deepF.8.', None, None),
    ((r'(branch1\.)convt_F19\.',), 'convt_F19.', 'deepF.9.', None, None),
    ((r'(branch2\.)SA1\.',), 'SA1.', 'deepF.1.', None, None),
    ((r'(branch2\.)convt_F11\.',), 'convt_F11.', 'deepF.2.', None, None),
    ((r'(branch2\.)convt_F12\.',), 'convt_F12.', 'deepF.3.', None, None),
    ((r'(branch2\.)convt_F13\.',), 'convt_F13.', 'deepF.4.', None, None),
    ((r'(branch2\.)convt_F14\.',), 'convt_F14.', 'deepF.5.', None, None),
    ((r'(branch2\.)convt_F15\.',), 'convt_F15.', 'deepF.6.', None, None),
    ((r'(branch2\.)convt_F16\.',), 'convt_F16.', 'deepF.7.', None, None),
    ((r'(branch2\.)convt_F17\.',), 'convt_F17.', 'deepF.8.', None, None),
    ((r'(branch2\.)convt_F18\.',), 'convt_F18.', 'deepF.9.', None, None),
    ((r'(branch2\.)convt_F19\.',), 'convt_F19.', 'deepF.10.', None, None),
    ((r'(branch3\.)convt_F11\.',), 'convt_F11.', 'deepF.0.', None, None),
    ((r'(branch3\.)convt_F12\.',), 'convt_F12.', 'deepF.1.', None, None),
    ((r'(branch3\.)convt_F13\.',), 'convt_F13.', 'deepF.2.', None, None),
    ((r'(branch3\.)convt_F14\.',), 'convt_F14.', 'deepF.3.', None, None),
    ((r'(branch3\.)convt_F15\.',), 'convt_F15.', 'deepF.4.', None, None),
    ((r'(branch3\.)convt_F16\.',), 'convt_F16.', 'deepF.5.', None, None),
    ((r'(branch3\.)convt_F17\.',), 'convt_F17.', 'deepF.6.', None, None),
    ((r'(branch3\.)convt_F18\.',), 'convt_F18.', 'deepF.7.', None, None),
    ((r'(branch3\.)convt_F19\.',), 'convt_F19.', 'deepF.8.', None, None),
    ((r'(branch3\.)convt_F20\.',), 'convt_F20.', 'deepF.9.', None, None),
    ((r'(branch.\.deepF\..*\.)scale1',), 'scale1', 'scale.0.scale', None, None),
    ((r'(branch.\.deepF\..*\.)scale2',), 'scale2', 'scale.1.scale', None, None),
    ((r'(branch.\.deepF\..*\.)scale3',), 'scale3', 'scale.2.scale', None, None),
    ((r'(branch.\.deepF\..*\.)scale4',), 'scale4', 'scale.3.scale', None, None),
    ((r'(branch.\.deepF\..*\.)scale5',), 'scale5', 'scale.4.scale', None, None),
    ((r'(branch.\.)SA2\.',), None, None, 'SA2', 'combineF.'),
    ((r'branch.\.u1\.',), 'conv.', '0.', None, None),
    ((r'branch.\.u1\.',), 'prelu.', '2.', None, None),
    ((r'branch.\.u2\.',), 'conv.', '0.', None, None),
    ((r'branch.\.u2\.',), 'prelu.', '2.', None, None),
    ((r'(branch.\.)u1\.',), None, None, 'u1', 'combineF.'),
    ((r'(branch.\.)conv_input\.',), None, None, 'conv_input.', 'inputF.'),
    ((r'(branch.\.)relu\.',), None, None, 'relu.', 'inputF.'),
    ((r'()branch3\.',), None, None, 'branch3.', 'U.3.3.'),
    ((r'()SA3\.',), None, None, 'SA3.', 'U.3.3.'),
    ((r'()branch2\.',), None, None, 'branch2.', 'U.3.'),
    ((r'()SA2\.',), None, None, 'SA2.', 'U.3.'),
    ((r'()down2_2\.',), None, None, 'down2_2.', 'U.3.'),
    ((r'()branch1\.',), None, None, 'branch1.', 'U.'),
    ((r'()down2_1\.',), None, None, 'down2_1.', 'U.'),
]

In [ ]:
removeRoot(weights, 'branch3.cats')
removeRoot(weights, 'branch3.u2')
removeRoot(weights, 'branch1.SA1')
removeRoot(weights, 'branch1.conv_input2')
for scale3, scale4 in getW(((r'branch.\.deepF\..*.scale.',), '3.scale', '4.scale', None, None)):
    weights[scale3] *= weights[scale4]
ccW(rsts)
m.load_state_dict(weights)

In [ ]:
w1 = m.state_dict()
list(w1.keys())

In [ ]:
torch.save(model.state_dict(), modelPath)

In [ ]:
paths = reduce(lambda a, t: a + [t[0] + '/' + n for n in t[2]], os.walk('../model'), [])
paths